In [1]:
import copy
import json
import math
import scipy

In [2]:
ranked_data_path = f"bigresponse.json"
ranked_data_file = open(ranked_data_path,encoding="utf-8")
ranked_data = json.load(ranked_data_file)

In [3]:
players = ranked_data["players"]
maps = ranked_data["maps"]
scores = ranked_data["scores"]
filtered_scores = [score for score in scores if score["modifiers"] == "" and score["accuracy"] < 100]

In [4]:
players_by_id = {player["id"] : player for player in players}
maps_by_id = {bmap["id"] : bmap for bmap in maps}

In [5]:
scores_by_player_id = {}
scores_by_map_id = {}

def add_score(score):
    player_id = score["playerId"]
    map_id = score["leaderboardId"]
    
    player_map = scores_by_player_id.get(player_id,{})
    player_map[map_id] = score
    
    scores_by_player_id[player_id] = player_map
    
    map_map = scores_by_map_id.get(map_id,{})
    map_map[player_id] = score
    
    scores_by_map_id[map_id] = map_map
    
#for score in scores:
for score in filtered_scores:
    add_score(score)

In [16]:
def clamp(v,vmin=0,vmax=500):
    return min(max(v,vmin),vmax)

In [7]:
"""
This is a mapping on the probability space rather than the population space.
That is, from (0,1) to (0,1). A probability distribution on the (0,1) interval.
We can do this with the Beta distribution, though other distributions might work.

I approximate a value for alpha and beta using this: https://homepage.divms.uiowa.edu/~mbognar/applets/beta.html
Trying to get the median around 0.9 and the probability close to 0 for 0.5, but a little bit higher for 0.65ish.
Beat Saber scores are typically at least 65%, on median probably around 0.9. They also must go down as the value
approaches 1, so beta must be greater than 1.

alpha = 10 and beta = 1.25 seemed to work quite well.
"""
beta_alpha = 10
beta_beta = 1.25

def beta_value(perc_value, beta_alpha=beta_alpha, beta_beta=beta_beta):
    return scipy.stats.beta.cdf(perc_value, beta_alpha, beta_beta)

def perc_beta_value(beta_value,beta_alpha=beta_alpha, beta_beta=beta_beta):
    return scipy.stats.beta.ppf(beta_value, beta_alpha, beta_beta)

"""
After renormalizing with the beta distribution, we apply the probit value, which assumes it's centred around 0.5.

We renormalize this to a value that is more typical of what we usually understand in Beat Saber.
For example, similar to star values. This is not really meant to be accurate, though.
"""

probit_mean = 7
probit_sd = 3

def probit_value(perc_value, probit_mean=probit_mean, probit_sd = probit_sd):
    return scipy.stats.norm.ppf(beta_value(perc_value),loc=probit_mean,scale=probit_sd)

def perc_probit_value(probit_value, probit_mean=probit_mean, probit_sd = probit_sd):
    return perc_beta_value(scipy.stats.norm.cdf(probit_value,loc=probit_mean,scale=probit_sd))

In [8]:
"""
We take the beta renormalization of probabilities above, and instead of using a probit function,
we map it to (0,inf) using a lognorm distribution.

There isn't a strong justification for this (other than the support), but there are some arguments.
What we want is to transform the probabilities into positive values that we can then multiply and divide in a way
that behaves well. Multiplying two values in the lognorm support produces a new lognorm value that is
the result of combining the other two in an independent way.
The shape of the lognorm also matches the way that we expect these values to look.
"""

# This parameterization is NOT a mistake. Read up on lognorm parameters and try to figure out why I did this.
# It isn't something terribly meaningful.
lognorm_mean = 7
lognorm_sd = 7
log_lognorm_sd = math.log(math.log(lognorm_sd))

def lognorm_value(perc_value, lognorm_mean=lognorm_mean, lognorm_sd=lognorm_sd, log_lognorm_sd=log_lognorm_sd):
    return scipy.stats.lognorm.ppf(beta_value(perc_value),s=log_lognorm_sd,loc=0,scale=lognorm_mean)

def perc_lognorm_value(lognorm_value, lognorm_mean=lognorm_mean, lognorm_sd=lognorm_sd, log_lognorm_sd=log_lognorm_sd):
    return perc_beta_value(scipy.stats.lognorm.cdf(lognorm_value,s=log_lognorm_sd,loc=0,scale=lognorm_mean))

In [9]:
"""
Very similar idea to the above, but with a maximum value.

There are some issues with this, like that a 100% score could lower a player's skill level
if it is on a very easy map. But this is very unlikely with the numbers, and 100% scores
are exceedingly rare anyway. There are also ways to deal with that later after estimating
difficulties.
"""

truncexp_max = 25
truncexp_base_mean = 7

def truncexp_value(perc_value,base_mean=truncexp_base_mean):
    return scipy.stats.truncexpon.ppf(beta_value(perc_value),b=truncexp_max,scale=base_mean)

def perc_truncexp_value(truncexp_value,base_mean=truncexp_base_mean):
    return perc_beta_value(scipy.stats.truncexpon.cdf(truncexp_value,b=truncexp_max,scale=base_mean))

In [10]:
"""
From here onwards we assume score is linearized by using linear_score above.

We normalize around a value of 7.
This means that a X star player on a 7 accability map achieves an X star score
and a 7 star player on an X accability map achieves an X star score.
"""

linear_mean = 7
min_value = 0.00001

def score_linear(skill,accability,linear_mean = linear_mean):
    return skill * accability / linear_mean

def accability_linear(skill,score,linear_mean = linear_mean):
    if (skill < min_value):
        vskill = min_value
    else:
        vskill = skill
    return score / vskill * linear_mean

def skill_linear(accability,score,linear_mean = linear_mean):
    if (accability < min_value):
        vaccability = min_value
    else:
        vaccability = accability
    return score / vaccability * linear_mean

In [11]:
"""
The main idea here is that the score of a player with skill level x on a map with accability x will be x.
In other words, the skill and accability are on the same scale as the score.
A player with skill level x will obtain score x on a map of their corresponding accability. Etc.
"""
def score_equalized(skill,accability):
    return math.sqrt(skill * accability)

def accability_equalized(skill,score):
    return score*score / skill

def skill_linear(accability,score):
    return score*score / accability

In [12]:
"""
The idea above is not completely broken. The main issue lies in the relationship with the score
being direct, which means that if one of skill / accability is larger than the other,
the resulting score expected is closer to the lower one. That is, this "mean" is lower
than the arithmetic mean, whereas in practice it should be the other way around (I think?).
If we have values between 0 and 1, we can fix this by subtracting 1 from them.

On top of that, this works best for values between 0 and 1, which are not linear when combining multiple scores.
So what we do here is put them in probability space before combining them, then combine them
there by subtracting them from 1.
"""
def score_probspace(skill, accability, value_func, perc_func):
    return value_func(1 - math.sqrt((1 - perc_func(skill))*(1 - perc_func(accability))))

def accability_probspace(skill, score, value_func, perc_func):
    perc_score = perc_func(score)
    inv_perc_score = 1 - perc_score
    
    return value_func(1 - (inv_perc_score*inv_perc_score)/(1 - perc_func(skill)))

def skill_probspace(accability, score, value_func, perc_func):
    perc_score = perc_func(score)
    inv_perc_score = 1 - perc_score
    
    return value_func(1 - (inv_perc_score*inv_perc_score)/(1 - perc_func(accability)))

def score_probspace_truncexp(skill, accability):
    return score_probspace(skill, accability, truncexp_value, perc_truncexp_value)

def accability_probspace_truncexp(skill, score):
    return accability_probspace(skill, score, truncexp_value, perc_truncexp_value)

def skill_probspace_truncexp(accability, score):
    return skill_probspace(accability, score, truncexp_value, perc_truncexp_value)

In [13]:
def abs_prop_error(value,evalue):        
    if (value == 0):
        vvalue = 0.0001
    else:
        vvalue = value
    return abs((vvalue-evalue)/vvalue)

In [14]:
class BiPartiteStabilizer:
    """No description yet"""
    
    def __init__(self,anodes_ratings,bnodes_ratings,wdata,afun,bfun,wfun,default_rating=1,max_iter=50,error_fun=abs_prop_error):
        """
        anodes_ratings and bnodes_ratings must be dictionaries with the node identifiers as keys
        and initial ratings as values.
        
        wdata must be a doubly indexed dictionary with anode identifiers and bnode identifiers as respective indexes,
        respectively, and weights as values.
        
        afun and bfun must be functions taking a value of the other node type as first argument
        and a weight as the second argument, that returns the corresponding value for the node.
        
        Similarly, wfun must be a function that takes the value of an anode and bnode and returns the
        correct weight.
        
        These functions must be such that:
        - wfun(afun(b,w),b) = w
        - wfun(a,bfun(a,w)) = w
        - afun(bfun(a,w),w) = a
        - afun(b,wfun(a,b)) = a
        - bfun(afun(b,w),w) = b
        - bfun(a,wfun(a,b)) = b
        
        error_fun must take two parameters (actual value, expected value) and return a number indicating the error for that
        particular edge
        """
        self.anodes_ratings = anodes_ratings
        self.bnodes_ratings = bnodes_ratings
        
        self.adata = wdata
        self.bdata = self.process_bdata(wdata)
        
        self.afun = afun
        self.bfun = bfun
        self.wfun = wfun
        
        self.error_fun = error_fun
        
        self.default_rating = default_rating
        
        self.average_error = math.inf
        
        self.iter = 0
        self.max_iter = max_iter
    
    def process_bdata(self, wdata):
        bdata = {}
        
        for anode_id, anode_data in wdata.items():
            for bnode_id, w in anode_data.items():
                bnode_data = bdata.get(bnode_id,{})
                bnode_data[anode_id] = w
                bdata[bnode_id] = bnode_data
        
        return bdata       
        
    def acycle(self):
        aerror = 0
        n = len(self.anodes_ratings)
        for anode_id in self.anodes_ratings:
            aerror += self.anode_process(anode_id)
            
        if (n > 0):
            self.average_error = clamp(aerror/n)
        else:
            self.average_error = 0
    
    def anode_process(self,anode_id):
        anode_data = self.adata[anode_id]
        
        asum = 0
        n = len(anode_data)
        for bnode_id, w in anode_data.items():
            bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
            asum += clamp(self.afun(bnode_value,w))
        
        if (n > 0):
            avg = asum/n
        else:
            avg = 0
        
        self.anodes_ratings[anode_id] = clamp(avg)
                                              
        return self.anode_error(anode_id)
        
    def anode_error(self,anode_id):
        anode_data = self.adata[anode_id]
        anode_value = self.anodes_ratings.get(anode_id,self.default_rating)
        
        aerror = 0
        n = len(anode_data)
        for bnode_id, w in anode_data.items():
            bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
            calculated_w = clamp(self.wfun(anode_value,bnode_value))
            error = clamp(self.error_fun(calculated_w,w))
            aerror += error
            
        if (n > 0):
            return clamp(aerror/n)
        else:
            return 0

    def bcycle(self):
        for bnode_id in self.bnodes_ratings:
            self.bnode_process(bnode_id)
    
    def bnode_process(self,bnode_id):
        bnode_data = self.bdata[bnode_id]
        
        bsum = 0
        n = len(bnode_data)
        for anode_id, w in bnode_data.items():
            anode_value = self.anodes_ratings.get(anode_id,self.default_rating)
            bsum += clamp(self.bfun(anode_value,w))
        
        if (n > 0):
            avg = bsum/n
        else:
            avg = 0
        
        self.bnodes_ratings[bnode_id] = clamp(avg)
    
    def iterate(self):
        self.bcycle()
        self.acycle()
        
        self.iter += 1
        
        print(f"Iteration {self.iter}, Average error: {self.average_error}")
        
    def run(self):
        while (self.iter <= self.max_iter):
            self.iterate()
        
        return (self.anodes_ratings,self.bnodes_ratings)
    
    def test(self):
        return

In [15]:
#scores_doubly_indexed = {map_id : {player_id : clamp(lognorm_value(scores_by_map_id[map_id][player_id]["accuracy"])) for player_id in scores_by_map_id[map_id]} for map_id in scores_by_map_id}
scores_doubly_indexed = {map_id : {player_id : truncexp_value(scores_by_map_id[map_id][player_id]["accuracy"]) for player_id in scores_by_map_id[map_id]} for map_id in scores_by_map_id}

In [17]:
default_rating = 7

map_ratings = {map_id : default_rating for map_id in scores_by_map_id}
player_ratings = {player_id: default_rating for player_id in scores_by_player_id}

bps = BiPartiteStabilizer(map_ratings,player_ratings,scores_doubly_indexed,accability_linear,skill_linear,score_linear,max_iter=7)
#bps = BiPartiteStabilizer(map_ratings,player_ratings,scores_doubly_indexed,accability_probspace_truncexp,skill_probspace_truncexp,score_probspace_truncexp,max_iter=7)

#bps.iterate()
#(maps_1,players_1) = (copy.deepcopy(bps.anodes_ratings), copy.deepcopy(bps.bnodes_ratings))
#bps.iterate()
#(maps_2,players_2) = (copy.deepcopy(bps.anodes_ratings), copy.deepcopy(bps.bnodes_ratings))
(map_ratings,player_ratings) = bps.run()

Iteration 1, Average error: 0.8327359680483292
Iteration 2, Average error: 0.9774494680197023
Iteration 3, Average error: 1.0866453888182588
Iteration 4, Average error: 1.1637818208373554
Iteration 5, Average error: 1.2417819821088403
Iteration 6, Average error: 1.3333611247847872
Iteration 7, Average error: 1.4258855466793152
Iteration 8, Average error: 1.5030894919325006


In [ ]:
map_outliers = {map_id : rating for (map_id,rating) in map_ratings.items() if rating > 20}
player_outliers = {player_id : rating for (player_id,rating) in player_ratings.items() if rating > 20}

In [ ]:
print(map_outliers)
print(player_outliers)

In [ ]:
hard_maps = {map_id : rating for (map_id,rating) in map_ratings.items() if rating < 6}
print(hard_maps)

In [ ]:
easy_maps = {map_id : rating for (map_id,rating) in map_ratings.items() if rating > 28}
print(easy_maps)

In [ ]:
good_players = {player_id : rating for (player_id,rating) in player_ratings.items() if rating > 20}
print(good_players)

In [ ]:
#print(scores_doubly_indexed["7e8f11"])
#print(maps_by_id["387a0xxxx91"])
print(players_by_id["76561198167588802"])

In [ ]:
print(map_ratings["3aa79xxxxxxxxx91"])

In [ ]:
filtered_map_ratings = {map_id : rating for (map_id,rating) in map_ratings.items() if rating < 3.69 and rating > 3.68}
print(filtered_map_ratings)

In [ ]:
print(len(scores))
print(len(filtered_scores))

In [ ]:
print(scores_by_player_id["76561198167588802"])

In [ ]:
print(truncexp_value(0.94))
print(perc_truncexp_value(12))

In [ ]:
print(skill_probspace_truncexp(0,25))

In [ ]:
print(score_probspace_truncexp(68,68))

In [ ]:
print(skill_linear(1,25))